# ***Fonctions utiles***


---
## Partie 1

### 1. Import des librairies (ok)
---

In [1]:
# Selon le projet ces librairies pourront êtres ajustées en fonction des besoins (ajout, suppression) - J'essaie de couvrir la majorité des besoins
# Nppt :
    # Faire un tri dans toutes ces lib et les ranger correctement pour un meilleur visu
    # Mettre les libs non utiles en commentaire pour les garder sous la main au cas ou
# Librairies de bases
import pandas as pd
import numpy as np
import sklearn as sk
import scipy as sp
import matplotlib
import statsmodels

# Scipy pour stats
from scipy.stats import pearsonr # Coeff de pearson
from scipy import stats


# StatModel
import statsmodels.api as sm
import statsmodels.formula.api as smf

# SKlearn pour PCA et clustering et autre
from sklearn.decomposition import PCA # Pour faire une ACP
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler # Scalling des données
from sklearn.metrics import silhouette_samples, silhouette_score, confusion_matrix
from sklearn.cluster import KMeans # Clustering KMeans
from sklearn import preprocessing, decomposition
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import AgglomerativeClustering # Clustering CAH
from sklearn.metrics import f1_score # Pour le calcul du F1-Score des modèles

# Librairie pour les graphs et autres
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.cm as cm
import seaborn as sns
sns.set_style('darkgrid', {'grid.color': '.5', 'grid.linestyle': ':'}) # Défini un fond de graphe que ce soit pour sns ou plt

# Librairies diverses
from statsmodels.stats.outliers_influence import variance_inflation_factor # Calcul de la corrélation entre nos variables choisies (Variance Inflation Factor)
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import  GridSearchCV # Recherche de param pour modèles de ML

    # Librairie nous permettant de faire une Régression Linéaire et Logistique
from sklearn.model_selection import train_test_split # Split des données pour train des modeles
from sklearn.linear_model import LogisticRegression # Regression logistique SKLearn
from sklearn.linear_model import LinearRegression # Regression linéaire SKLearn
    # Metrics pour évaluation de modèles ML
from sklearn.metrics import roc_auc_score , roc_curve, accuracy_score , precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score


    # Librairies nécessaire pour réaliser une matrice de confusion
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Pour stocker des fonctions à part et les réutiliser
import joblib

#### *Vérification import lib et run des fct*

In [2]:
# Test pour voir si le book des fonctions est bien lancé et les libs bien chargées
# Affiche la version des librairies chargées
def fct_test():
    """
    Vérification de l'importation des librairies et du run du book de fonctions
    Affichage des versions des librairies chargées
    
    
    return :
    -----------------------------------------------------------------------------------------------
        Version des librairies chargées
        Message de confirmation du chargement des librairies et des fonctions
    """
    
    print('Librairies utilisées :')
    print('+-------------------------------------+')
    print('    Pandas :', pd.__version__)
    print('     Numpy :', np.__version__)
    print('Matplotlib :', matplotlib.__version__)
    print('   Seaborn :', sns.__version__)
    print('     Spicy :', sp.__version__)
    print('Statmodels :', statsmodels.__version__)
    print('   Sklearn :', sk.__version__)
    print('    JobLib :', joblib.__version__)
    print('+-------------------------------------+')
    print('')

#### *Divers*

In [3]:
# Longeur max pour infos sur fonction : ----------------------------------------------------------------------------------------------- (95 caractères depuis première marge)

---
## Partie 2

### 1. Fonctions utiles
---

#### *Infos de bases*

In [4]:
# Première infos sur un dataset, info(), isna(), describe()
def fct_first_look(df):
    """
    Première visualisation du df
    Renvoi des informations de bases comme .info(), .describe() ou encore le nombre de manquants par variables
    
    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    arg :
        df (pandas.DataFrame) - DataFrame contenant les données à analyser 
    
    return :
    -----------------------------------------------------------------------------------------------
        display .info(), .isna().sum() et .describe(include = 'all')
    """
    
    print('Visu du dataset (entête) :')
    display(df.sample(4))
    print('')
    print('+-----------------------------------+')
    print('Infos sur le df :')
    display(df.info())
    print('')
    print('+-----------------------------------+')
    print('Nombre de valeurs NaN')
    display(df.isna().sum())
    print('')
    print('+-----------------------------------+')
    print("Déscription du df (include = 'all')")
    display(round(df.describe(include = 'all'), 4))

#### *Triangle de corrélations*

In [5]:
# Identique qu'une Heatmap mais visualisation différente
def fct_corr_matrix(df, annot = True):
    """
    Affiche la matrice de corrélation entre les données, reprend la heatmap mais la découpe
    en forme de triangle
    
    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    arg :
        df (pandas.DataFrame) - DataFrame contenant les données à analyser 
    
    return :
    -----------------------------------------------------------------------------------------------
        Triangle de corrélation entre les variables
    """
    
    # Matrice de corrélation
    matrix = df.corr()

    # Triangle de corrélations
    mask = np.triu(np.ones_like(matrix, dtype = bool))
    plt.figure(figsize=(12, 6))
    
    plt.title('Triangle des corrélations', fontdict = {'fontsize' : '14', 'color' : 'black', 'fontweight' : 'bold'})
    sns.heatmap(matrix, mask = mask, annot = annot, cmap = 'RdGy')
    
    plt.show()

#### *Heatmap*

In [6]:
# Identique que triangle de corrélation mais visualisation complète
def fct_corr_heatmap(data, annot=True):
    """
    Calcule le coefficient de corrélation de Pearson entre toutes les paires de variables d'un df
    
    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    arg :
        df (pandas.DataFrame) - DataFrame contenant les données à analyser
    
    return :
    -----------------------------------------------------------------------------------------------
        Heatmap contenant les coefficients de corrélation entre toutes les paires de variables
    """
    
    plt.figure(figsize=(12,6))
    
    plt.title("Heatmap de corrélation de Pearson", fontdict = {'fontsize' : '14', 'color' : 'black', 'fontweight' : 'bold'})
    sns.heatmap(data.corr(), annot = annot, cmap = 'RdGy')
    
    plt.show()

#### *Scaling des données*

In [7]:
# Scale les variables quantitative selon méthode sélectionnée
def fct_data_scaler(df, columns = None, method = 'standard', scaler_all = False):
    """
    Scale les données d'un DataFrame en utilisant différentes méthodes de scaling,
    possibilité de sélection des colonnes
    
    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    arg :
        df (pd.DataFrame) : DataFrame contenant les données à scaler
        columns (list): Liste des noms de colonnes à scaler. Si aucune liste n'est spécifiée,
        toutes les colonnes sont scalées (par défaut: None)
        
        method (str): Méthode de scaling à utiliser. Les valeurs possibles sont 'standard' (par défaut),
        'minmax', 'robust' et 'log'
        
        scaler_all (bool): Si True, scaler toutes les colonnes du DataFrame (par défaut: False)
    
    return :
    -----------------------------------------------------------------------------------------------
        pd.DataFrame : DataFrame contenant les données scalées
    """
    
    # On vérifie si on doit appliquer le scaler sur toutes nos données
    if columns is None and not scaler_all:
        raise ValueError("Spécifiez les colonnes à scaler ou activez l'option 'scaler_all' pour scaler toutes les colonnes.")
    
    # Si scaller_all == True, on travail sur toutes les colonnes
    if scaler_all:
        columns = df.columns.tolist()
    
    # On sélectionne la méthode de Scalage et on l'applique à nos données.
    if method == "standard":
        scaler = StandardScaler()
    elif method == "minmax":
        scaler = MinMaxScaler()
    elif method == "robust":
        scaler = RobustScaler()
    elif method == "log":
        def log_scaler(data):
            return np.log1p(data)
        scaler = log_scaler
    else:
        raise ValueError("Méthode de scaling non valide. Les valeurs possibles sont 'standard', 'minmax', 'robust' et 'log'.")

    # On scale nos données avec la méthode sélectionnée    
    if method == "log":
        df_scaled = df[columns].apply(scaler)
    else:
        scaler.fit(df[columns])
        scaled_data = scaler.transform(df[columns])
        df_scaled = pd.DataFrame(scaled_data, columns = columns, index = df.index)
    
    # On remplace nos données initiales par les données scalées tout en conservant les données que nous ne voulions pas traiter
    df = pd.concat([df.drop(columns, axis=1), df_scaled], axis=1)
    
    return df

#### *IQR sur une variable*

In [8]:
# Fonction de recherche des outliers avec la méthode des IQR
# En cours de dévelloppement
# Source : From Scratch
def fct_outliers_iqr(data,  disp = True):
    """
    data (pd.series or np.ndarray) :
        Colonne d'un df ou nd.array contenant les données à traiter
    
    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    data (pd.series or np.array) :
        Variable à tester
    disp (bool) :
        Affiche ou non le df avec les outliers (défaut = True)
        
    return :
    -----------------------------------------------------------------------------------------------
        outlier ((pd.DataFrame) :
            Dataframe contenant la liste des outliers et leurs index
        
        La liste des index des outliers
        Un dataframe listant tous les outliers dépassant 1.5x IQR (si disp = True)
        lower_bound, upper_bound
    """
    
    # Faire un test des données d'entrées (test sur le type)
    
    # On calcule l'interquartile (on pourrait passer un arg pour définir selon besoin)
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1
    print('Quartilles :')
    print('')
    print("Q1 => " + str(round(q1,  4)))
    print("Q3 => " + str(round(q3, 4)))
    print("IQR => " + str(round(iqr, 4)))
    
    # Calcul des valeurs limites (on pourrait passer un arg pour les définir selon besoin)
    lower_bound = q1 -(1.5 * iqr)
    upper_bound = q3 +(1.5 * iqr)
    print("Lower_bound => " + str(round(lower_bound, 4)))
    print("Upper_bound => " + str(round(upper_bound, 4)))

    # Outliers
    print('')
    print('--------------------------------------------------------------')
    print('Liste des outliers selon la méthode des écarts interquartilles')
    print('--------------------------------------------------------------')
    print('')
    outlier = data[(data > upper_bound) | (data < lower_bound)]
    # On passe le résultat sous forme de dataframe
    outlier = pd.DataFrame(outlier)
    
    # A améliorer pour ne pas avoir de double affichage parfois
    if disp == True:
        display(outlier)
    
    return outlier.index.tolist(), lower_bound, upper_bound

#### *Variance Inflation Factor*

In [9]:
# Source : From Scratch
def fct_vif(data):
    """
    Effectue un VIF sur les données du dataframe

    Parameters : 
    -----------------------------------------------------------------------------------------------
    data :
        pandas.DataFrame : DataFrame contenant les données quantitatives
    
    return :
    -----------------------------------------------------------------------------------------------
        pandas.DataFrame : DataFrame contenant les données VIF
        Affichage uniquement
    
    Infos :
    -----------------------------------------------------------------------------------------------
    /!\ Le dataframe d'entrée ne doit pas contenir de valeurs NaN, de str ou de bool
    
    Dépendance linéaire entre les variables :
    
    If the degree of correlation is high enough between variables, it can cause problems when
    fitting and interpreting the regression model
    
    A general rule of thumb for interpreting VIFs is as follows :
    
        A value of 1 indicates there is no correlation between a given explanatory variable and
        any other explanatory variables in the model
    
        A value between 1 and 5 indicates moderate correlation between a given explanatory variable
        and other explanatory variables in the model,but this is often not severe enough to
        require attention
        
        A value greater than 5 indicates potentially severe correlation between a given explanatory
        variable and other explanatory variables in the model
        
        One way to detect multicollinearity is by using a metric known as the
        variance inflation factor (VIF), which measures the correlation and strength of
        correlation between the explanatory variables in a regression model
    """
    
    # Test sur le type de variables à traiter
    for i in data.columns:
        # Test sur type de variable
        if data[i].dtypes != 'float64' and data[i].dtypes != 'int64':
            print('Type non supporté :', data[i].dtypes)
            print('-------------------------------------')
            print(data.dtypes)
            print('Erreur, veuillez vérifier les données')
            raise ValueError('Le df contient des données non numériques...')
            break
        
        # Test des NaN
        elif data[i].isna().sum() != 0:
            print('Nombre de NaN :', data[i].isna().sum())
            print('-------------------------------------')
            print(data.isna().sum())
            print('Erreur, veuillez vérifier les données')
            raise ValueError('Le df contient des données manquantes...')
            break
        
        # Validation de la colonne   
        else:
            print('')
            print('Variable vérifiée :', i)
            print('Ok pour traitement des données après vérifcation...')
    
    # VIF dataframe
    vif_data = pd.DataFrame() # On créé un df vide
    vif_data['feature'] = data.columns # On prend les colonnes de notre df
      
    # Calcul du VIF pour chaque feature
    vif_data['VIF'] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]
    
    # df de sortie avec les valeurs VIF
    print('')
    print('+---------------------------------+')
    print('| VIF - Variance Inflation Factor |')
    print('+---------------------------------+')
    display(vif_data)
    # Faire une rapide explication du VIF
    

#### *Shapiro - Test de normalité*

In [10]:
# Source : From Scratch
def fct_shapiro(data, graph = True, bins = 20, color = 'Red'):
    """
    Effectue un test de shapiro afin de vérifier si la distribution des variables suit une
    loi normale
    Test possible sur plusieurs colonnes d'un pd.DataFrame mais ne renverra que le graphique

    Parameters : 
    -----------------------------------------------------------------------------------------------
    data (pandas.DataFrame or np.ndarray) :
        Array of sample data, possible de passer un df complet ou certaines colonnes avec [[,]]
    graph (bool) :
        Choix d'afficher le graphique ou non, par défaut = True
    bins (int) :
        Spécifie le nombre de bins pour l'histogramme, par défaut = 20
    color (str) :
        Permet de choisir la couleur du graphique
    
    return :
    -----------------------------------------------------------------------------------------------
        tuple containing :
            statistic : float
                The test statistic
            p-value : float
                The p-value for the hypothesis test
        
        sns.histplot : Graphique de la distribution de la variable
    
    Infos :
    -----------------------------------------------------------------------------------------------
    Test Shapiro, test pour vérifier la normalité d'UNE variable

        H0 : la distribution suit une loi normale
        H1 : la distribution ne suit pas une loi normale

        Si p-value > 0.05, hypothèse H0 acceptée, distribution normale
        Si p-value < 0.05, hypothèse H0 rejectée, distribution pas normale
        
        Si plusieurs variables sont passées en entrée, n'éffectue pas le test mais retourne un
        histogramme avec les différences variables
    """
    
    
    
    # Génération du graphique de la distribution sous forme d'histogramme
    if graph == True:
        plt.figure(figsize=(16,6))
        
        sns.histplot(data = data, kde = True, color = color, bins = bins)
        plt.title('Histogramme de distribution', fontdict = {'fontsize' : '14', 'color' : 'black', 'fontweight' : 'bold'})
        
        plt.show()
    
    # Si ce n'est pas une série n'affichera pas les infos du test Shapiro
    # Print des résultats si Shapiro sur une seule variable
    if type(data) ==  pd.core.series.Series or type(data) == np.ndarray:
        # On effectue le test et on récupère les valeurs retournées
        # Récupération de stats
        stat_shapiro = stats.shapiro(data)[0]
        # Récupération de la p-value
        pvalue_shapiro  = stats.shapiro(data)[1]
        
        print('Résultats du test Shapiro :')
        print('-----------------------------------------------------------------------------------------------')
        print('Statistique :', round(stat_shapiro, 3))
        print('p-value :', pvalue_shapiro)
        print("Nombre d'individus :", len(data), "(Attention, si nombre d'individus élevés)")
        print('-----------------------------------------------------------------------------------------------')
    
        # Analyse des résultats
        print('')
        print('Note :')
        print('-----------------------------------------------------------------------------------------------')
        print('H0 : la distribution suit une loi normale')
        print('H1 : la distribution ne suit pas une loi normale')
        print('')
        print('Si p-value > 0.05, hypothèse H0 acceptée, distribution normale')
        print("Si p-value < 0.05, hypothèse H0 rejetée, distribution n'est pas normale")
        print('')

        if pvalue_shapiro < 0.05:
            print('p-value < 0.05 : H0  Rejetée, la distribution ne suit pas une loi normale')
        else:
            print('p-value > 0.05 : H0 acceptée, la distribution suit une loi normale')
        
    elif  type(data) !=  pd.core.series.Series or type(data) != np.ndarray:
        # Récupération de stats
        stat_shapiro = 0
        # Récupération de la p-value
        pvalue_shapiro  = 0
        
        print('Trop de variables')
    
    # Ajouter une info sur le nombre d'individus testé pour l'appréciation du test
    
    # return des résultats sous forme de tuple si besoin
    return stat_shapiro, pvalue_shapiro

In [11]:
# Test de Shapiro sur toutes les colonnes d'un dataframe
# En cours de création, l'idée est de faire un shapiro sur chaque variables passée en entrée

def fct_shapiro_multi(data, graph = True, bins = 20, hue = None, color = 'DarkRed'):
    """
    Affiche la distribution de chaque variable du dataframe passé en argument
    Pour analyser une variable particulière utiliser la fonction :
        fct_shapiro(data, graph = True, summary = True)

    Parameters : 
    -----------------------------------------------------------------------------------------------
    data (pandas.DataFrame) :
        Array of sample data, possible de passer un df complet ou certaines colonnes avec [[,]]
    graph (bool) :
        Choix d'afficher le graphique ou non, par défaut = True    
    bins (int) :
        Spécifie le nombre de bins pour l'histogramme, par défaut = 20
    color (str) :
        Permet de choisir la couleur du graphique
    
    return :
    -----------------------------------------------------------------------------------------------        
        sns.histplot : Graphique de la distribution de la variable
    """
    
    #  En premier tester les données pour savoir si possible de générer le graph
    
    # Histogramme de la distribution des valeurs par variables (si graph = True)
    if graph == True:
        for i in data.columns:
            fig, ax = plt.subplots(1, 1, figsize = (12, 2))
            
            sns.histplot(ax = ax, data = data, x = i, kde = True, fill = True, hue = hue, bins = bins, color = color)
            
            plt.show()
            
            # On effectue le test et on récupère les valeurs retournées
            # Récupération de stats
            stat_shapiro = stats.shapiro(data)[0]
            # Récupération de la p-value
            pvalue_shapiro  = stats.shapiro(data)[1]
            
            print('Résultats du test Shapiro :', i)
            print('-----------------------------------------------------------------------------------------------')
            print('Statistique :', stat_shapiro)
            print('p-value :', pvalue_shapiro)
            print("Nombre d'individus :", len(data), "(Attention, si nombre d'individus élevés)")
            print('-----------------------------------------------------------------------------------------------')
            
    return

#### *ACP - Algo de calcul*

In [12]:
# Calcul les composantes de l'ACP et le scree_cum, pour le cercle de corrélation et la projection des individus, utiliser les fonction à la suite après avoir fait l'ACP
# Penser à scaler les données avant et à faire attention aux NaN

In [13]:
# Source : From Scratch
def fct_pca(data, n_comp = 2):
    """
    Effectue une ACP sur les données

    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    data :
        pandas.DataFrame : DataFrame contenant les données scalées
    n_comp :
        int : nombre de composantes de l'ACP (par défaut : 2)
    
    return :
    -----------------------------------------------------------------------------------------------
        ACP entrainée sur les données
        Eboulis des valeurs propres
        Heatmap
        Retourne un tuple :
            x[0] c'est l'objet pca
            x[1] tableau pcs (composantes par variables)
            x[2] c'est la projection des individus (composantes par individus)
            x[3] c'est le df complet avec la projection
    """
    
    # Faire un module de test des paramètres d'entrée
    # Pas de NaN, de bool ou de str dans quaque colonne du df d'entrée
    # Scaler les données (faire une option qui le fait ou non)
    
    
    # Nombre de composantes voulues, par defaut = 2
    n_components = n_comp
    
    # On instancie notre PCA avec le nombre de composantes voulues
    pca = PCA(n_components = n_comp)
    # On l'entraine sur notre df donné en entrée
    pca.fit(data)
    
    # On récupère la variance expliquée
    scree = (pca.explained_variance_ratio_*100).round(2)
    scree_cum = scree.cumsum().round()
    
    # On fait une liste des composantes calculées
    x_list = range(1, n_comp + 1)
    list(x_list)
    
    # Graphique d'éboulis des valeurs propres
    plt.bar(x_list, scree, color = 'Grey')
    plt.plot(x_list, scree_cum, c = 'red', marker = 'o')
    
    plt.xlabel("Rang de l'axe d'inertie")
    plt.ylabel("Pourcentage d'inertie")
    plt.title("Eboulis des valeurs propres", fontdict = {'fontsize' : '14', 'color' : 'black', 'fontweight' : 'bold'})
    
    plt.show(block=False)
    
    # Nombre de composantes qui expliquent la variance
    print(len(scree_cum), 'composantes expliquent', scree_cum[-1], '% de la variance')
    
    # Visu des composantes
    pcs = pca.components_
    # On met les composantes sous forme de df
    pcs = pd.DataFrame(pcs)
    features = data.columns
    # On regarde les composantes par variables
    pcs.columns = features
    pcs.index = [f"F{i}" for i in x_list]
        
    # Heatmap sur les composantes de l'ACP
    print('+---------------------------------------------------------------------------------------------+')
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(pcs.T, vmin=-1, vmax=1, annot=True, cmap="RdGy", fmt="0.2f")
    plt.title('Heatmap Composantes / Features', fontdict = {'fontsize' : '14', 'color' : 'black', 'fontweight' : 'bold'})
    plt.show()
    
    
    # Calcul des coordonnées de projection par individus
    # On entraine et on transforme sur df_scaled que l'on met dans X_proj
    X_proj = pca.fit_transform(data)
    X_proj[:5]
    
    # On fait un df des composantes de nos individus sur les différents axes en vue de les rajouter dans notre CSV final
    df_projection = pd.DataFrame(X_proj)
    df_projection.columns = [f'F{i}' for i in x_list]
    df_projection = df_projection.set_index(data.index)
    df_projection
    
    # Faire un code qui retourne un df avec : df de base + projection des variables à la suite
    df = pd.DataFrame(data)
    df = pd.merge(left = df, right = df_projection, left_index = True, right_index = True, how = 'inner')
    
    # Retourne un tuple : x[0] c'est la pca, x[1] c'est les composantes par variables, x[2] c'est les composantes par individus, x[3] c'est le df complet avec la projection
    return pca, pcs.T, df_projection, df
    

#### *ACP - Cercle de corrélation*

In [14]:
# Fonction pour cercle de corrélation
# Il faut avoir fait l'ACP au préalable puis la passer en argument

In [15]:
# Source : TP OCR - Adaptation : DsFx
def fct_correlation_graph(pca, x_y, features): 
    """
    Affiche le graphe des correlations

    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    pca :
        sklearn.decomposition.PCA : notre objet PCA qui a été fit sur nos data (quantitatives et scalées de préférence)
        ou
        reprendre ce qui est retourné par fct_pca sous forme xx[0] = fct_pca(data, n_comp = 2)
    x_y :
        list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2
    features :
        list ou tuple : la liste des features (ie des dimensions) à représenter
        ou
        reprendre les colonnes du df passé dans l'arg pca ou xx[1].index
    
    return :
    -----------------------------------------------------------------------------------------------
        graphique du cercle des corrélations
    """

    # Extrait x et y 
    x,y=x_y

    # Taille de l'image (en inches)
    fig, ax = plt.subplots(figsize=(10, 9))

    # Pour chaque composante : 
    for i in range(0, pca.components_.shape[1]):

        # Les flèches
        ax.arrow(0,0, 
                pca.components_[x, i],  
                pca.components_[y, i],  
                head_width=0.07,
                head_length=0.07, 
                width=0.02, )

        # Les labels
        plt.text(pca.components_[x, i] + 0.05,
                pca.components_[y, i] + 0.05,
                features[i])
        
    # Affichage des lignes horizontales et verticales
    plt.plot([-1, 1], [0, 0], color='grey', ls='--')
    plt.plot([0, 0], [-1, 1], color='grey', ls='--')

    # Nom des axes, avec le pourcentage d'inertie expliqué
    plt.xlabel('F{} ({}%)'.format(x+1, round(100*pca.explained_variance_ratio_[x],1)))
    plt.ylabel('F{} ({}%)'.format(y+1, round(100*pca.explained_variance_ratio_[y],1)))

    # J'ai copié collé le code sans le lire, et c'est tout à fait mon genre bien sur ;)
    plt.title("Cercle des corrélations (F{} et F{})".format(x+1, y+1), fontdict = {'fontsize' : '14', 'color' : 'black', 'fontweight' : 'bold'})

    # Le cercle 
    an = np.linspace(0, 2 * np.pi, 100)
    plt.plot(np.cos(an), np.sin(an))  # Add a unit circle for scale

    # Axes et display
    plt.axis('equal')
    plt.show(block=False)

#### *ACP - Projection de l'ACP*

In [16]:
# Fonction pour projection des individus sur les axes

In [17]:
# Source : TP OCR - Adaptation : DsFx
def fct_display_factorial_planes(X_projected, x_y, pca=None, labels = None, clusters=None, alpha=1, figsize=[10,8], marker=".", palette = 'OrRd'):
    """
    Affiche la projection des individus

    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    X_projected :
        np.array, pd.DataFrame, list of list : la matrice des points projetés,
        recup du tuple de fct_pca[2]
    x_y :
        list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2

    Optional arguments : 
    -----------------------------------------------------------------------------------------------
    pca : sklearn.decomposition.PCA : un objet PCA qui a été fit, cela nous permettra d'afficher
    la variance de chaque composante, default = None
    
    labels : list ou tuple : les labels des individus à projeter, default = None
    
    clusters : list ou tuple : liste des clusters auquel appartient chaque individu, default = None
    
    alpha : float in [0,1] : transparence, 0=100% transparent, 1=0% transparent, default = 1
    
    figsize : list ou tuple : couple width, height qui définit la taille de la figure en inches,
    default = [10,8] 
    
    marker : str : le type de marker utilisé pour représenter les individus, points croix etc etc,
    default = "."
    
    palette (str) : Le type de palette utilisé (défaut = 'OrRd')
    
    return :
    -----------------------------------------------------------------------------------------------
        graphique de la projection des individus
    """

    # Transforme X_projected en np.array
    X_ = np.array(X_projected)

    # On définit la forme de la figure si elle n'a pas été donnée
    if not figsize: 
        figsize = (7,6)

    # On gère les labels
    if  labels is None : 
        labels = []
    try : 
        len(labels)
    except Exception as e : 
        raise e

    # On vérifie la variable axis 
    if not len(x_y) ==2 : 
        raise AttributeError("2 axes sont demandées")   
    if max(x_y )>= X_.shape[1] : 
        raise AttributeError("la variable axis n'est pas bonne")   

    # on définit x et y 
    x, y = x_y

    # Initialisation de la figure       
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    # On vérifie s'il y a des clusters ou non
    c = None if clusters is None else clusters
 
    # Les points    
    # plt.scatter(   X_[:, x], X_[:, y], alpha=alpha, 
    #                     c=c, cmap="Set1", marker=marker)
    sns.scatterplot(data=None, x=X_[:, x], y=X_[:, y], hue=c, palette = palette)

    # Si la variable pca a été fournie, on peut calculer le % de variance de chaque axe 
    if pca : 
        v1 = str(round(100*pca.explained_variance_ratio_[x]))  + " %"
        v2 = str(round(100*pca.explained_variance_ratio_[y]))  + " %"
    else : 
        v1=v2= ''

    # Nom des axes, avec le pourcentage d'inertie expliqué
    ax.set_xlabel(f'F{x+1} {v1}')
    ax.set_ylabel(f'F{y+1} {v2}')

    # Valeur x max et y max
    x_max = np.abs(X_[:, x]).max() *1.1
    y_max = np.abs(X_[:, y]).max() *1.1

    # On borne x et y 
    ax.set_xlim(left=-x_max, right=x_max)
    ax.set_ylim(bottom= -y_max, top=y_max)

    # Affichage des lignes horizontales et verticales
    plt.plot([-x_max, x_max], [0, 0], color='grey', alpha=0.8)
    plt.plot([0,0], [-y_max, y_max], color='grey', alpha=0.8)

    # Affichage des labels des points
    if len(labels) : 
        # j'ai copié collé la fonction sans la lire... Hahaha, oui, c'est tout a fait mon genre mdr (merci le petit easter egg OCR ;) )
        # Cette fonction me reservira plus tard surement, elle fini dans ma cheat sheet ^^
        for i,(_x,_y) in enumerate(X_[:,[x,y]]):
            plt.text(_x, _y+0.05, labels[i], fontsize='10', ha='center', va='center') 

    # Titre et display
    plt.title(f"Projection des individus (sur F{x+1} et F{y+1})", fontdict = {'fontsize' : '14', 'color' : 'black', 'fontweight' : 'bold'})
    
    plt.show()

#### *Clustering CAH -A faire*

In [18]:
# Dendrogramme
# Diagramme d'inertie

#### *Clustering KMeans - A faire*

In [19]:
# Méthode du coude
# Score de silhouette

#### *Grid Search*

#### *Regression Linéaire*

#### *Regression logistique*

#### *KNN*

#### *Recherche de variables explicatives pour une régression linéaire multi*

In [20]:
# Source : TP OCR - Adaptation : DsFx
def fct_back_select_linear(data, response, summary = False):
    """
    Linear model designed by backward selection

    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    data (pd.DataFrame) :
        DataFrame avec toute les variables et la réponse  (valeur à prédire)

    response (str) :
        Nom de la colonne de réponse dans data, ce qui est conservé
    
    summary (bool) :
        Affiche les différentes étapes du process de sélection

    return :
    -----------------------------------------------------------------------------------------------
    model: modèle "optimal" entrainé avec statsmodel
           sélection par élimination des paramètres non-significatifs (backward selection)
           évaluation par p-value
           
    Infos :
    -----------------------------------------------------------------------------------------------
    Fonction qui va analyser les variables explicatives en fonction de la variable recherchée
    Au fur et à mesure va supprimer des variables afin de ne conserver que les paramètres
    significatifs
    """
    
    print('Recherche de variables explicatives pertinentes pour :', response)
    print('')
    
    # Retire la colonne 'response' et défini le paramètre de sortie de la boucle
    remaining = set(data._get_numeric_data().columns)
    if response in remaining:
        remaining.remove(response)
    cond = True
    
    # Boucle qui va tester les variables au fur et à mesure
    while remaining and cond:
        # On prend les différentes variables
        formula = "{} ~ {} + 1".format(response,' + '.join(remaining))
        # Déroulement de la sélection par suppression des paramètres non-significatifs
        print('_______________________________')
        print(formula)
        # Train de ols
        model = smf.ols(formula, data).fit()
        # Récupération des p-values
        score = model.pvalues[1:]
        
        # Recherche de la p-values la plus élevée
        # Si p-values > à 0.05, le paramètre est retiré
        # Si toutes les p-values sont < 0.05, on arrête la boucle
        
        if summary == True:
            print(model.summary()) # Pour suivi de l'évolution de recherche, peut être passé en commentaire si bcp de paramètres à tester
        
        toRemove = score[score == score.max()]
        if toRemove.values > 0.05:
            # Résumé de la recherche
            print('remove', toRemove.index[0], '(p-value :', round(toRemove.values[0],3), ')')
            remaining.remove(toRemove.index[0])
        else:
            # Donne le modèle final le plus pertinent
            cond = False
            print('')
            print('+============================================================================+')
            print('|                        Le modèle final !  (summary)                        |')
            print('+============================================================================+')
        print('')
    # Information sur le modèle
    print(model.summary())
    
    return model

#### *Recherche de variables explicatives pour une régression logistique*

In [21]:
# Source : TP OCR - Adaptation : DsFx (la fonction smf. change uniquement)
def fct_back_select_logit(data, response, summary = False):
    """
    Logistic model designed by backward selection

    Positional arguments : 
    -----------------------------------------------------------------------------------------------
    data (pd.DataFrame) :
        DataFrame avec toute les variables et la réponse (valeur à prédire)

    response (str) :
        Nom de la colonne de réponse dans data, ce qui est conservé
    
    summary (bool) :
        Affiche les différentes étapes du process de sélection

    return :
    -----------------------------------------------------------------------------------------------
    model: modèle "optimal" entrainé avec statsmodel
           sélection par élimination des paramètres non-significatifs (backward selection)
           évaluation par p-value
           
    Infos :
    -----------------------------------------------------------------------------------------------
    Fonction qui va analyser les variables explicatives en fonction de la variable recherchée
    Au fur et à mesure va supprimer des variables afin de ne conserver que les paramètres
    significatifs
    """
    
    print('Recherche de variables explicatives pertinentes pour :', response)
    print('')
    
    # Retire la colonne 'response' et défini le paramètre de sortie de la boucle
    remaining = set(data._get_numeric_data().columns)
    if response in remaining:
        remaining.remove(response)
    cond = True
    
    # Boucle qui va tester les variables au fur et à mesure
    while remaining and cond:
        # On prend les différentes variables
        formula = "{} ~ {} + 1".format(response,' + '.join(remaining))
        # Déroulement de la sélection par suppression des paramètres non-significatifs
        print('_______________________________')
        print(formula)
        # Train de ols
        model = smf.logit(formula, data).fit()
        # Récupération des p-values
        score = model.pvalues[1:]
        
        # Recherche de la p-values la plus élevée
        # Si p-values > à 0.05, le paramètre est retiré
        # Si toutes les p-values sont < 0.05, on arrête la boucle
        
        if summary == True:
            print(model.summary()) # Pour suivi de l'évolution de recherche, peut être passé en commentaire si bcp de paramètres à tester
        
        toRemove = score[score == score.max()]
        if toRemove.values > 0.05:
            # Résumé de la recherche
            print('remove', toRemove.index[0], '(p-value :', round(toRemove.values[0],3), ')')
            remaining.remove(toRemove.index[0])
        else:
            # Donne le modèle final le plus pertinent
            cond = False
            print('')
            print('+============================================================================+')
            print('|                        Le modèle final !  (summary)                        |')
            print('+============================================================================+')
        print('')
    # Information sur le modèle
    print(model.summary())
    
    return model

#### *Vérification lancement des fonctions*

In [22]:
# Cette fonction doit rester à la fin de ce notebook et être exécutée après le chargement de celui dans un notebook afin d'être sur qu'il soit importer en mémoire
def fct_load():
    print('')
    print('+--o=      Test run DsFx_fct      =o--+')
    print('')
    print('+-------------------------------------+')
    print('|   Import lib : OK - Loaded          |')
    print('|      Run fct : OK - Loaded          |')
    print('+-------------------------------------+')
    print('                                       ')
    print('_______________________________________')
    print(" _    __    _                          ")
    print("| \ _|_   _|___|_  o  | _  _  _| _  _| ")
    print("|_/_>|><___|(_ |_  o  |(_)(_|(_|(/_(_| ")
    print(" _                                 ... ")
    print("|_) _  _  _|\/ _|_ _      _ _      ||| ")
    print("| \(/_(_|(_|/   |_(_) |_|_>(/_     000 ")
    print('_______________________________________')

In [23]:
# Coding by : David GESSER
# 28/12/2023